# 2.0 Merge dataset 
### feature engineering

Procederemos a mergear los dos datasets trabajados en el data wrangling. 
- df_wine_clean.csv
- df_wine_spa_clean.csv
  

## Import libraries

In [1]:
# Sirve para actualizar el modulo despues de haber cambiado algo en el codigo de los modulos importados
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
%matplotlib inline
# Modulo para trabajar con los directorios del proyecto
from src.utils.paths import data_dir
from src.visualization.word_cloud import words_cloud
from src.visualization.analysis_univariate import univariate_analysis
import src.utils.statistics as stats
from src.utils.paths import references_dir
from src.visualization.word_cloud import words_cloud_maskable_image

## Read Data

In [3]:
# Obtenemos la ruta de los csv's que vamos a utilizar
WINE_CLEAN_CSV_PATH = data_dir("intermediate", "df_wine_clean.csv")
WINES_SPA_CLEAN_CSV_PATH = data_dir("intermediate", "df_wine_spa_clean.csv")

WINE_CLEAN_CSV_PATH , WINES_SPA_CLEAN_CSV_PATH

(WindowsPath('c:/Users/xpam08/Desktop/SOFI/CODERHOUSE/DATA SCIENCE/vinos-corderhouse/data/intermediate/df_wine_clean.csv'),
 WindowsPath('c:/Users/xpam08/Desktop/SOFI/CODERHOUSE/DATA SCIENCE/vinos-corderhouse/data/intermediate/df_wine_spa_clean.csv'))

### SETUP

In [4]:
# Leemos los csv's y los guardamos en un dataframe
df_wine= pd.read_csv(WINE_CLEAN_CSV_PATH)
df_wine_spa = pd.read_csv(WINES_SPA_CLEAN_CSV_PATH)

In [5]:
print(f"El shape de df_wine es: {df_wine.shape}")
print(f"El shape de df_wine_spa es: {df_wine_spa.shape}")

El shape de df_wine es: (5013, 13)
El shape de df_wine_spa es: (5558, 8)


Como se comento en notebooks anteriores, ambos datasets se obtuvieron de distintas fuentes, por lo que no habra columna comun en ambos para utilizarla para mergear, por lo que trabajaremos con los indices de ambos dataframes.

Para eso necesitamos equiparar la cantidad de filas de ambos datasets.
Como df_wine_spa es el que mas filas tiene, le elminaremos la diferencia de filas.

In [6]:
difference = df_wine_spa.shape[0] - df_wine.shape[0] 
print(f"La diferencia de filas entre df_wine y df_wine_spa es: {difference}")

La diferencia de filas entre df_wine y df_wine_spa es: 545


## Equiparar los dataframes

Cuales 545 registros sacamos, puede ser muy interesante. En este caso elegirimos sacar las filas cuyos valores en las columnas categoricas sean los menos frecuentes

In [7]:
# Declaramos una lista vacia para almacenar los indices de los outliers a eliminar
index_rows_to_drop = []

### Winery

In [8]:
# Verificamos los valores unicos de una columna categorica que tenemos: winery de df_wine_spa
winery_counts = df_wine_spa['winery'].value_counts().sort_values(ascending=True)
winery_counts

Cal Batllet - Celler Ripoll Sans      1
Castell del Remei                     1
Valdubon                              1
Loess                                 1
Olivardots                            1
                                   ... 
Losada                              206
Artadi                              211
Sierra Cantabria                    212
La Rioja Alta                       213
Contino                             410
Name: winery, Length: 368, dtype: int64

In [9]:
#Obtenemos la lista de los 30 winery que solo tengan 3 o menos registros
winery_less_common = winery_counts[winery_counts <= 4]
df_wine_spa_less_common_winery = df_wine_spa[df_wine_spa['winery'].isin(winery_less_common.index)]
df_wine_spa_less_common_winery.shape

(508, 8)

In [10]:
# Considerando estos registros los agregamos a la lista de filas que vamos a eliminar
index_rows_to_drop.extend(df_wine_spa_less_common_winery.index)
print(f"La cantidad de columnas a eliminar hasta el momento es de {len(set(index_rows_to_drop))}")
print(f"La cantidad de filas que faltan eliminar es de {difference - len(set(index_rows_to_drop))}")

La cantidad de columnas a eliminar hasta el momento es de 508
La cantidad de filas que faltan eliminar es de 37


### Wines

In [11]:
# Verificamos los valores unicos de una columna categorica que tenemos: wine de df_wine_spa
wine_counts = df_wine_spa['wine'].value_counts().sort_values(ascending=True)
wine_counts

Juan Piernas Finca El Paso Malo Monastrell      1
Nivel                                           1
Cerro Las Cuevas Seleccion Terroir              1
Graciano                                        1
Diego Magana                                    1
                                             ... 
El Viejo                                      202
Corimbo I                                     202
Rioja Reserva                                 214
Gran Reserva                                  407
Reserva                                       419
Name: wine, Length: 579, dtype: int64

In [12]:
#Obtenemos la lista de los winery que solo tengan menos de 2 registros
wine_less_common = wine_counts[wine_counts <= 2]
df_wine_spa_less_common_wine = df_wine_spa[df_wine_spa['wine'].isin(wine_less_common.index)]
df_wine_spa_less_common_wine.shape

(557, 8)

Como nos pasamos de la diferncia para equiparar los dataframes, restamos lo que nos falta


In [13]:
# identificamos la cantidad de indices que no estan en index_rows_to_drop
temp_index = df_wine_spa_less_common_wine.index.difference(index_rows_to_drop)
print(f"La cantidad de indices que encontramos y que no estan en index_rows_to_drop es {len(temp_index)}")

La cantidad de indices que encontramos y que no estan en index_rows_to_drop es 164


In [14]:
# Determinamos cuanto es lo que nos falta para completar la diferencia de filas
how_much_left = difference - len(set(index_rows_to_drop))
how_much_left

37

In [15]:
# Modificamos df_wine_spa_less_common_wine a la cantidad maxima de filas que faltan 
max_quantity_index_drop = how_much_left if temp_index.size > how_much_left else temp_index
df_wine_spa_less_common_wine = df_wine_spa_less_common_wine.loc[temp_index].head(max_quantity_index_drop)
df_wine_spa_less_common_wine.shape

(37, 8)

In [16]:
# Considerando estos registros los agregamos a la lista de filas que vamos a eliminar
index_rows_to_drop.extend(df_wine_spa_less_common_wine.index)
print(f"La cantidad de columnas a eliminar hasta el momento es de {len(set(index_rows_to_drop))}")
print(f"La cantidad de filas que faltan eliminar es de {difference - len(set(index_rows_to_drop))}")


La cantidad de columnas a eliminar hasta el momento es de 545
La cantidad de filas que faltan eliminar es de 0


In [17]:
#Eliminamos las filas que contienen outliers y reseteamos el index
print(f"La cantidad de columnas a eliminar es de {len(set(index_rows_to_drop))}")
df_wine_spa.drop(index=index_rows_to_drop, inplace=True, axis=0)
df_wine_spa.reset_index(drop=True, inplace=True)
print("Dataframe limpio")
print(f"{df_wine_spa.shape}")


La cantidad de columnas a eliminar es de 545
Dataframe limpio
(5013, 8)


In [18]:
# Por ultimo guardamos el dataframe limpio en un archivo csv
WINE_SPA_EQUAL_CSV_PATH = data_dir("intermediate", "df_wine_spa_equal.csv")
df_wine_spa.to_csv(WINE_SPA_EQUAL_CSV_PATH, index=False)

# Merge

In [19]:
df_merged = pd.merge(df_wine_spa, df_wine, left_index=True, right_index=True)
print(f"El shape de df_merged es: {df_merged.shape}")
df_merged.sample(5)

El shape de df_merged es: (5013, 21)


,winery,wine,year,rating,region,price,type,body,fixed_acidity,volatile_acidity,...,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,style
4219,Matarromera,Gran Reserva,2011,4.2,Ribera del Duero,64.50,Ribera Del Duero Red,5.0,6.2,0.32,...,2.9,0.029,37.0,94.0,0.98998,3.25,0.60,12.4,6,white
850,Conreria d'Scala Dei,Les Brugueres,2018,4.2,Priorato,16.76,Priorat Red,4.0,6.8,0.28,...,8.0,0.045,28.0,123.0,0.99280,3.02,0.37,11.4,6,white
3277,Clos Pons,Roc Nu,2011,4.2,Costers del Segre,23.95,Red,4.0,6.8,0.32,...,9.6,0.026,34.0,124.0,0.99286,3.18,0.35,12.1,6,white
1627,Clos Pons,Roc Nu,2011,4.2,Costers del Segre,23.95,Red,4.0,6.3,0.23,...,5.1,0.035,29.0,142.0,0.99420,3.36,0.33,10.1,7,white
3915,Enrique Mendoza,Santa Rosa,2017,4.2,Alicante,18.90,Red,4.0,7.0,0.22,...,9.2,0.027,37.0,122.0,0.99228,3.06,0.34,12.5,8,white


Podemos corroborar que el merge se hizo correctamente porque coinciden la suma de la cantidad de columnas y cantidad de registros de ambos dataframes.

In [20]:
# Por ultimo guardamos el dataframe mergeado en un archivo csv
WINE_MERGE_CSV_PATH = data_dir("intermediate", "df_wine_merged.csv")
df_merged.to_csv(WINE_MERGE_CSV_PATH, index=False)


In [21]:
df_merged = pd.read_csv(data_dir("intermediate", "df_wine_merged.csv"))
df_merged.shape

(5013, 21)